# Processing document using default Standard output

# Introduction

Amazon Bedrock Data Automation (BDA) lets you configure output based on your processing needs for a specific data type: documents, images, video or audio. BDA can generate standard output or custom output.

You can use standard outputs for all four modalities: documents, images, videos, and audio. BDA always provides a standard output response even if it's alongside a custom output response.

Standard outputs are modality-specific default insights, such as video summaries that capture key moments, visual and audible toxic content, explanations of document charts, graph figure data, and more. 

In this notebook we will explore the Standard output for the documents.

## Prerequisites

In [1]:
pip install "boto3>=1.35.76" PyPDF2 --upgrade

Note: you may need to restart the kernel to use updated packages.


## Setup

Before we get to the part where we invoke BDA with our sample artifacts, let's setup some parameters and configuration that will be used throughout this notebook

In [27]:
import boto3
import json
import pprint
from IPython.display import JSON
import IPython.display as display


region_name = 'us-west-2'
# Initialize Bedrock Data Automation client
bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

bda_s3_input_location = 's3://genai.octankmarkets.com/input'
bda_s3_output_location = 's3://genai.octankmarkets.com/output'

## Prepare Sample Document
For this lab, we use a `Monthly Treasury Statement for the United States Government` for Fiscal Year 2025 through November 30, 2024. The document is prepared by the Bureau of the Fiscal Service, Department of the Treasury and provides detailed information on the government's financial activities. We will extract a subset of pages from the `PDF` document and use BDA to extract and analyse the document content.

### Download and store sample document
we use the document url to download the document and store it a S3 location. 

Note - We will configure BDA to use the sample input from this S3 location, so we need to ensure that BDA has `s3:GetObject` access to this S3 location. If you are running the notebook in your own AWS Account, ensure that the SageMaker Execution role configured for this JupyterLab app has the right IAM permissions.

In [30]:
%load_ext autoreload
%autoreload 2

from utils.helper_functions import wait_for_job_to_complete, read_s3_object, create_sample_file, get_bucket_and_key

# Download sample pdf file, extract pages and  
input_bucket, input_prefix = get_bucket_and_key(bda_s3_input_location)
sample_data_url = "https://fiscaldata.treasury.gov/static-data/published-reports/mts/MonthlyTreasuryStatement_202411.pdf"
local_file_name = 'examples/MonthlyTreasuryStatement_202411.pdf'
create_sample_file(sample_data_url, 0, 9, local_file_name)
s3_file_name = 'MonthlyTreasuryStatement.pdf'
s3_response = s3_client.upload_file(local_file_name, input_bucket,
                                    f'{input_prefix}/{s3_file_name}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### View Sample Document

In [36]:
display.IFrame("examples/MonthlyTreasuryStatement_202411.pdf", width=900, height=800)

## Standard output in Bedrock Data Automation

We start with document data type. Here we use a sample credit report which is of type `pdf`. Below is a summary of the response options that you can set when using standard output with documents. 

For more details see [Documents](https://docs.aws.amazon.com/bedrock/latest/userguide/bda-output-documents.html).

- **Response granularity**
This setting indicates to BDA the kind of response you want to receive from document text extraction. Each level of granularity gives you more and more seperated responses, with page providing all of the text extracted together, and word providing each word as a seperate response. The available granularity levels are:

    - Page
    
    - Element
    
    - Word

    More details [Response Granularity](https://docs.aws.amazon.com/bedrock/latest/userguide/bda-output-documents.html#document-granularity)

- **Output settings**
Output settings determine the structure of the results produced by BDA. The options for output settings are:

    - **JSON** - The result would be a JSON output file with the information from your configuration settings. This is the **default** for document analysis.
    
    - **JSON+files**  The result would include a JSON output along with files that correspond with different outputs. For example, this setting gives you a text file for the overall text extraction, a markdown file for the text with structural markdown, and CSV files for each table that's found in the text.

- **Text Format**
Text format determines the different kinds of texts that will be provided via various extraction operations. You can select any number of the following options for your text format.

    - **Plaintext** – This setting provides a text-only output with no formatting or other markdown elements noted.
    
    - **Text with markdown** – The **default** output setting for standard output. Provides text with markdown elements integrated.
    
    - **Text with HTML** – Provides text with HTML elements integrated in the response.
    
    - **CSV** – Provides a CSV structured output for tables within the document. This will only give a response for tables, and not other elements of the document

- **Bounding Boxes**

    - With the Bounding Boxes option enabled, BDA would output `Bounding Boxes` for elements in the document in form of coordinates of four corners of the box. This helps in creating a visual outline of the element in the document.


- **Generative Fields**
    
    - When `Generative Fields` are enabled, BDA generates a 10-word summary and a 250 word description of the document in the output. Additionally with Response Granularity at element level enabled, BDA also generates a descriptive caption of each figure detected in the document. Figures include things like charts, graphs, and images.


  Both these options are **disabled by default**.


Now that we have looked at the default options, we can proceed to invoking document processing using the default Standard output configuration.

<div class="alert alert-block alert-info">
<b>Note: TBC-Disclaimer</b> 
</div>

### Invoke Data Automation Async

In [4]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f'{bda_s3_input_location}/{s3_file_name}'
    },
    outputConfiguration={
        's3Uri': bda_s3_output_location
    }
)

invocationArn = response['invocationArn']

### Get Data Automation Status

We can check the status and monitor the progress of the Invocation job using the `GetDataAutomationStatus`. This API takes the invocation arn we retrieved from the response to the `InvokeDataAutomationAsync` operation above.

The invocation job status moves from `Created` to `InProgress` and finally to `Success` when the job completes successfully, along with the S3 location of the results. If the job encounters and error the final status is either `ServiceError` or `ClientError` with error details

In [8]:
status_response = wait_for_job_to_complete(invocationArn=invocationArn)
if status_response['status'] == 'Success':
    job_metadata_s3_location = status_response['outputConfiguration']['s3Uri']
else:
    raise Exception(f'Invocation Job Error, error_type={status_response["error_type"]},error_message={status_response["error_message"]}')

Invocation Job with id 552f296d-8af8-4fcd-92dc-4b034fb7a9e4 completed. Status is Success


### Retrieve Job Metadata

In [9]:
job_metadata = json.loads(read_s3_object(job_metadata_s3_location))
JSON(job_metadata,root='job_metadata',expanded=True)

<IPython.core.display.JSON object>

### Explore the default Standard output
We can now explore the standard output received from processing documents using Data Automation. 

The default Output Format is Json so the standard output for our sample document is in Json format. 

The Standard output for `Document` modality always includes a `metadata` section and a `document` section in the json result. Because for Standard output with default values, the Response granularity of Page level and Element level are enabled by default, the results would include sections for `page` and `element` level information.

We go through the different components in the response in the following sections. 

In [40]:
JSON(standard_output)

<IPython.core.display.JSON object>

In [10]:
asset_id=0
standard_output_path = next(item["segment_metadata"][0]["standard_output_path"] 
                                for item in job_metadata["output_metadata"] 
                                if item['asset_id'] == asset_id)
standard_output = json.loads(read_s3_object(standard_output_path))

### metadata
The metadata section in the response provides an overview of the metadata associated with the document. This include the S3 bucket and key for the input document. The metadata also contains the modality that was selected for your response, the number of pages processed as well as the start and end page index.

In [11]:
JSON(standard_output['metadata'],root='metadata',expanded=True)

<IPython.core.display.JSON object>

### document
The document section of the standard output provides document level granularity information. Document level granularity would include an analysis of information from the document providing key pieces of info.

By default the document level granularity includes statistics that contain information on the actual content of the document, such as how many semantic elements there are, how many figures, words, lines, etc. We will look at further information that would be presented in the document level granularity when we modify the standard output using projects.

In [37]:
JSON(standard_output['document'],root='document', expanded=True)

<IPython.core.display.JSON object>

### pages
With Page level granularity (enabled by default) text in a page are consolidated and are listed in the pages section with one item for each page. The page entity in the Standard output include the page index. The individual page entities also include the statistics that contain information on the actual content of the document, such as how many semantic elements there are, how many figures, words, lines, etc. The asset metadata represents the page bounds using coordinates of the four corners.

Below, we look at a snippet of the output pertaining to a specific page.

In [39]:
JSON(standard_output['pages'][0],root='pages[0]',expanded=False)

<IPython.core.display.JSON object>

### element
The element section contains the various semantic elements extracted from the documents including Text content, Tables and figures. The text and figure entites are further sub-classified for example TITLE/SECTION_TITLE for Text or Chart for figures.

#### TEXT element

In [16]:
JSON(standard_output['elements'][5],root='elements[5]', expanded=True)

<IPython.core.display.JSON object>

#### FIGURE element

In [19]:
JSON(standard_output, expanded=True)

<IPython.core.display.JSON object>

#### TABLE element

In [17]:
JSON(standard_output['elements'][27],root='elements[27]', expanded=True)

<IPython.core.display.JSON object>

## Clean Up
Let's delete the sample files that were downloaded locally and that uploaded to S3

In [25]:
## Delete S3 File

s3_client.delete_object(Bucket=input_bucket, Key=f'{input_prefix}/{s3_file_name}')

#Delete local file
import os
if os.path.exists(local_file_name):
    os.remove(local_file_name)	

## Conclusion

In this notebook we started with the default way of interacting with Amazon Bedrock Automation (BDA) by passing a sample document to the BDA API with no established blueprint or project. We then explored the default default standard output for the sample document.

## Next Steps
Standard output can be modified using projects, which store configuration information for each data type. In the next part of the workshop we would explore Bedrock data automation projects.